# Plots for local linearization

In [ ]:
using LinearAlgebra
using Plots

using KernelFunctions
using StatsBase
using Zygote
using Interpolations
using Measurements
using DifferentialEquations

using Distributions
using StatsPlots, KernelDensity

using ThreadsX

In [ ]:
# run this cell for publication quality plots, which however take more time to build and more ram
# It will also require a latex installation with some relevant packages.
# pgfplotsx()

In [ ]:
ts = range(-4.25, 4.252, length = 100)
f(x) = x*cos(x)

X = range(-3., 3., length = 9)
σ_n = 0.02
# y = f.(X) .+ sqrt(σ_n) * randn(length(X));
# hard-coded for reproducibility
y = [3.0559694723914066, 1.4627220455340493, -0.10111838683803989, -0.6447352014461587, -0.028773298414122316, 0.6236280365707595, 0.3615475328182163, -1.375141794854174, -3.079245779724556]

In [ ]:
lwd = 2
l = 1.1
ker = pskernel([1., l])

K = kernelmatrix(ker, X') + σ_n*I
Kchol = cholesky(K)
gp(x) = ker.(x, X)' * (Kchol \ y)
Ks(x) = ker.(x,x') - ker.(x, X') * (Kchol \ ker.(X, x'))

vargp(x) = ker(x,x) - ker.(x, X)' * (Kchol \ ker.(x, X))
stdgp(x) = sqrt( ker(x,x) - ker.(x, X)' * (Kchol \ ker.(x, X)))

# define derivatives
dker(t1, t2) = first(first(Zygote.gradient(t1 -> ker(t1, t2), t1)))
dgp(x) = dker.(x, X)' * (Kchol \ y)

In [ ]:
xsample = range(-4.25, 4.252, length = 30)

function exact_sample(xsample)
    m = gp.(xsample)
    ζ = randn(length(xsample))
    K = Ks(xsample)
    K = 0.5 * (K+ K') + 1e-12*I
    Kchol = cholesky(K)

    sample = m .+ Kchol.L*ζ

    # LinearInterpolation(xsample, sample)
    itp = interpolate(sample, BSpline(Cubic(Line(OnGrid()))))
    sitp = Interpolations.scale(itp, xsample)
    extrapolate(sitp, Flat())
    # extrapolate(sitp, NaN)
end

In [ ]:
The following are multipe setup cells with relevant functions needed in the rest of the notebook.

In [ ]:
vtmp(x) = ker.(x,x') - ker.(x, X') * (Kchol \ ker.(X, x'))

function linearized_eulerstep!(x, a, h, n;  lhist = 150)
    a[n] = dgp(x[n].val)

    m = x[n].val + h*gp(x[n].val)

    ldx = max(1, n-lhist)
    μo = getfield.(x[ldx:n], :val)

    K = vtmp(μo)
    Kbr = K[end, :] 
    vb = Kbr[end]

    ah = (1 .+ a[ldx+1:n-1] .* h)
    ahe = reverse(vcat(1., cumprod(reverse(ah))))

    cv = sum(ahe .* Kbr[1:end-1]) 
    v = (1+a[n]*h)^2 * x[n].err^2 + h^2 * vb + 2*h^2* cv * (1+a[n]*h) 

    x[n+1] = m ± sqrt(v)
end

function linearized_euler(x0, tspan, h; lhist = 150)
    nsteps = ceil(Int, diff(tspan)[1]/h)

    xe = zeros(Measurement{Float64}, nsteps+1)
    av = zeros(nsteps+1)

    xe[1] = x0 # μ₀ ± sqrt(Σ₀)

    for i in 1:nsteps
        linearized_eulerstep!(xe, av, h, i; lhist)
    end
    th = tspan[1]:h:h*nsteps

    th, xe
end

In [ ]:
function exp_step!(x, a, h, n; lhist)
    μ = x[n].val
    Σ = x[n].err^2

    a[n] = dgp( μ )

    # mean
    m = μ * exp(a[n]*h) - 1/a[n] * (1-exp(a[n]*h)) * (gp(μ) - a[n]*μ)

    # var
    ## setup
    ldx = max(1, n-lhist)
    μ_past = getfield.(x[ldx:n], :val)

    K = vtmp(μ_past)
    Kbr = K[end, :] 
    vb = Kbr[end]
    
    a_past = a[ldx:n-1]

    # can probably be more elegant
    if isempty(a_past) 
        cov = 0.
    else
        aex = exp.(cumsum(reverse(a_past[2:end])) .*h )
        ah = reverse(vcat(1., aex)) 
        bf =  (1 ./ a_past .* (1 .- exp.(a_past.*h)))

        cov = dot(ah, bf .* Kbr[1:end-1])
    end
    
    ## combine
    v = exp(2*a[n]*h) * Σ + vb/(a[n]^2) * (1 - exp(a[n]*h))^2 + 2 * exp(a[n]*h) * 1/a[n] * (1 - exp(a[n]*h)) * cov 

    x[n+1] = m ± sqrt(v)
end

function exp_solver(x0, tspan, h; lhist = 150)
    nsteps = ceil(Int, diff(tspan)[1]/h)

    xe = zeros(Measurement{Float64}, nsteps+1)
    av = zeros(nsteps+1)

    xe[1] = x0

    for i in 1:nsteps
        exp_step!(xe, av, h, i; lhist)
        # xe[i+1] = exp_step(xe[i], av, h, i; lhist)
    end
    th = tspan[1]:h:h*nsteps

    th, xe
end

In [ ]:
# this is very messy, really need to tie in with SparseGP objects

q(xi, w, μ_x, Σ_x) = ((1/w)*Σ_x + 1)^(-1/2)*exp(-1/2*(μ_x-xi)*(Σ_x+w)^(-1)*(μ_x-xi))
function mapp(μ_x, Σ_x, β, l, X)
    w = l^2
    qv = q.(X, Ref(w), Ref(μ_x), Ref(Σ_x))
    return dot(qv,β)
end

function Q(xi, xj, w, μ_x, Σ_x) 
    xb = (xi+xj)/2
    exp1 = exp(-1/2*(xb - μ_x)' * (1/2*w+Σ_x)^(-1) * (xb-μ_x))
    exp2 = exp(-1/2*(xi-xj)' * (2w)^(-1) * (xi-xj))
    ((2/w)*Σ_x + 1)^(-1/2) * exp1 * exp2
end

function vapp(μ_x, Σ_x, β, l, X, ker, Kchol)
    w = l^2
    Qm = Q.(X, X', Ref(w), Ref(μ_x), Ref(Σ_x)) # actually symmetric, might be potential to go faster
    v = ker(μ_x, μ_x) + tr(β*β'*Qm - Kchol\Qm) - mapp(μ_x, Σ_x, β, l, X)^2
    return v
end

function qh(xi, w, μ_x, Σ_x) 
    C = 1/(1/w + 1/Σ_x)
    ci = C*(xi/w + μ_x/Σ_x)
    
    return q(xi, w, μ_x, Σ_x)*ci
end

function covp(μ_x, Σ_x, β, l, X, ker, Kchol)
    w = l^2
    qhv = qh.(X, Ref(w), Ref(μ_x), Ref(Σ_x)) 
    cov = dot(qhv, β)
    
    return cov - μ_x*mapp(μ_x, Σ_x, β, l, X)
end

β = (Kchol \ y)

mc(μ, Σ) = mapp(μ, Σ, β, l, X)
vc(μ, Σ) = vapp(μ, Σ, β, l, X, ker, Kchol)
cc(μ, Σ) = covp(μ, Σ, β, l, X, ker, Kchol)

function eulerstep(x::Measurement{T}, h) where T <: Real
    μ = x.val
    Σ = x.err^2
    
    m = μ + h*mc(μ, Σ)
    v = Σ + h^2*vc(μ, Σ) + 2*h*cc(μ, Σ)
    
    return m ± sqrt(v)
end 

function solvevareuler(x0::Measurement{T}, tspan, h) where T<:Real
    nsteps = ceil(Int, (tspan[end] - tspan[1]) / h)+1
    vsol = zeros(nsteps) .± zeros(nsteps)
    
    vsol[1] = x0
    for i in 2:nsteps
        vsol[i] = eulerstep(vsol[i-1], h)
    end
    tsteps = range(tspan[1], tspan[2], length = nsteps)
    return tsteps, vsol
end

In [ ]:
# just to check that the code works 
let
    μ₀ = 0.6
    Σ₀ = 0.005

    tspan = [0, 12]
    
    h = 0.01
    th, xe = solvevareuler(μ₀ ± sqrt(Σ₀), tspan, h)

    plot(th, getfield.(xe, :val); ribbons = getfield.(xe, :err))
end

## Model

In [ ]:
let 
    p = plot(xlabel = "x", ylabel = "f(x)", legend = :topright)
    plot!(p, ts, gp.(ts), ribbons = 2*(stdgp.(ts)),
        color = :mediumseagreen, label = "GP mean + 2σ", linewidth = 2.9)

    greys = Plots.Colors.colormap("Grays", 9)[3:end-1]
    label = ""
    for i in 1:6
        samplf = exact_sample(xsample)
        if i == 6
            label = "samples"
        end
        plot!(p, ts, samplf.(ts); label, color = greys[i])
    end
    scatter!(p, X, y, color = :orange, markersize = 3.5, label = "data")
    global p_model = p
end

p_model
# 2;

In [ ]:
function getzeroprob(x)
    d = Normal(gp(x), stdgp(x))
    return pdf(d, 0.)
end

# let 
#     xv = range(0.7, 2.2, length = 400)
#     probs = getzeroprob.(xv)

#     w = FrequencyWeights(probs)

#     mz = mean(xv, w)
#     vz = std(xv, w)

#     global dzero = Normal(mz, vz)
#     display(dzero)

#     plot(xv, probs)
#     plot!(xv, pdf.(Ref(dzero), xv), label = "Identified")
# end

## Sample Solver

In [ ]:
## Sample based solver

μ₀ = 0.6
Σ₀ = 0.005

tspan = [0, 10]
nevalpoints = 101
evalpoints = range(tspan[1], tspan[2], length = nevalpoints)


nsols = 3000
ninputsmpls = 100
evalvals = 1.
enssols = 2.
function ens_samplepathsol()
    pathsmpl = exact_sample(xsample)
    f(x, p, t) = [pathsmpl(x[1])]
    prob = ODEProblem(f, [μ₀], tspan)

    function prob_func(prob, i, repeat)
        @. prob.u0 = μ₀ + randn()*sqrt(Σ₀)
        prob
    end
    
    ens_prob = EnsembleProblem(prob; prob_func)

    ens_sol = solve(ens_prob, Tsit5(), EnsembleSplitThreads(); trajectories = ninputsmpls, saveat = evalpoints);
end

let 
    @time global enssols = ThreadsX.map(x -> ens_samplepathsol(), 1:nsols);
    println("all done")

    # can be global, but huge, so we rather want to forget it after this cell
    @time evalvals = reduce(vcat, map( ens -> map(x -> reduce(vcat, getfield(x, :u)), ens), enssols))
    
    ldx = getindex.(evalvals, nevalpoints) .> 0.
    println(sum(.!ldx))
    evalvals = evalvals[ldx]
    sliceloc = 2.5
    global pidx = findfirst(evalpoints .> sliceloc)
    global kdest = kde(getindex.(evalvals, pidx))

    NS = 50
    global qs = reduce(hcat, [quantile.(Ref(getindex.(evalvals, i)), range(0.01, 0.99, NS)) for i in 1:nevalpoints])

    @time global m_in = [mean(getindex.(evalvals, i)) for i in 1:nevalpoints]
    @time global s_in = [std(getindex.(evalvals, i)) for i in 1:nevalpoints]

    global p1s = plot(evalpoints, m_in, ribbons = s_in,
    color = :mediumseagreen, label = "summary", linewidth = lwd, legend = :bottomright)
    global p2s = plot(evalpoints, s_in; label = "std", xlabel = "t")
    plot!(p2s, tspan, dzero.σ * ones(2); label = "fp std", color = :black)

    plot(p1s, p2s)
end

let 
    @time global enssols = ThreadsX.map(x -> ens_samplepathsol(), 1:nsols/2);
    println("all done")
    # enssols = [ens_samplepathsol() for i in 1:nsols];

    # can be global, but huge, so we rather want to forget it after this cell
    @time evalvals = reduce(vcat, map( ens -> map(x -> reduce(vcat, getfield(x, :u)), ens), enssols))

    ldx = getindex.(evalvals, nevalpoints) .> 0.
    println(sum(.!ldx))
    evalvals = evalvals[ldx]

    @time global m_in_low = [mean(getindex.(evalvals, i)) for i in 1:nevalpoints]
    @time global s_in_low = [std(getindex.(evalvals, i)) for i in 1:nevalpoints]

    plot!(p1s, evalpoints, m_in_low, ribbons = s_in_low,
    color = :mediumseagreen, label = "summary", linewidth = lwd, legend = :bottomright)
    plot!(p2s, evalpoints, s_in_low; label = "low sample std", xlabel = "t", linestyle = :dash)

    # 2;
    global ex_plot = plot(p1s, p2s)
end

enssols = nothing
evalvals = nothing
GC.gc()

ex_plot

In [ ]:
# linearized Euler results
hsp = diff(evalpoints)[1]
th_expt, xe_expt = linearized_euler(μ₀ ± sqrt(Σ₀), tspan, hsp; lhist = 700 )

In [ ]:
# bad method 
th, xe = solvevareuler(μ₀ ± sqrt(Σ₀), tspan, hsp)

In [ ]:
# # Inner plot prototype

# sliceloc = 2.5
# pidx = findfirst(evalpoints .> sliceloc)

# dcl = distinguishable_colors(1, [RGB(1,1,1), RGB(0,0,0), Plots.Colors.colormap("greens", 2)[2]], dropseed=true)

# kdest = kde(evalvals[pidx, :])
# nd = Normal(xe_expt[pidx].val, xe_expt[pidx].err)

# p = plot(legend = :topright)
# plot!(p, kdest, color = :black, label = "sample")
# plot!(p, nd, color = dcl[1], label = "approx.")

In [ ]:
let 
    NS = 50

    lbs = fill("", NS)
    lbs[ceil(Int, NS/4)] = "density"

    cls = Plots.Colors.colormap("greens", Int(NS/2))
    gcl = cls[end]
    cls = vcat(cls, reverse(cls))

    dcl = distinguishable_colors(1, [RGB(1,1,1), RGB(0,0,0), Plots.Colors.colormap("greens", 2)[2]], dropseed=true)

    p = plot(legend = :bottomright, xlabel = "time t", ylabel = "trajectory x(t)")
    for i in 1:NS
        plot!(p, evalpoints, qs[i, :]; color = cls[i], label = lbs[i], linewidth = 2)
    end

    plot!(p, evalpoints, m_in; label = "sample mean",
        color = :black, linewidth = 2.4)
    plot!(p, evalpoints, m_in .- s_in; label = "sample std dev",
        color = :grey35, linewidth = 1.9, linestyle = :dash)
    plot!(p, evalpoints, m_in .+ s_in; label = "",
        color = :grey35, linewidth = 1.9, linestyle = :dash)
    
    plot!(p, th_expt, getfield.(xe_expt, :val); label = "PULL Euler mean",
        color = dcl[1], linewidth = 2.4)
    plot!(p, th_expt, getfield.(xe_expt, :val) .- getfield.(xe_expt, :err); label = "PULL Euler std. dev.",
        color = dcl[1], linewidth = 1.9, linestyle = :dash)
    plot!(p, th_expt, getfield.(xe_expt, :val) .+ getfield.(xe_expt, :err); label = "",
        color = dcl[1], linewidth = 1.9, linestyle = :dash)

    sliceloc = 2.5
    apidx = findfirst(th_expt .> sliceloc)
    # pidx = findfirst(evalpoints .> sliceloc)
    # kdest = kde(evalvals[pidx, :])
    nd = Normal(xe_expt[apidx].val, xe_expt[apidx].err)

    plot!(p, evalpoints[pidx]*ones(2), qs[[1,end], pidx]; label = "", color = :grey50)

    inletxlim = (1.3, 1.9)
    plot!(p, kdest; inset = (1, bbox(0.32,0.22,0.66,0.41)), subplot = 2,
        color = gcl, label = "", xlims = inletxlim,
        ylabel = "probability", xlabel = "state x($sliceloc)")
    plot!(p, nd; color = dcl[1], label = "", subplot = 2)

    # plot(kdest, color = :black, label = "sample", legend = :topright)
    # plot!(nd, color = dcl, label = "approx.")


    global p_dens = p

    plot(p, size = (333, 350))
end


In [ ]:
let 
    NS = 50

    lbs = fill("", NS)
    lbs[ceil(Int, NS/4)] = "density"

    cls = Plots.Colors.colormap("greens", Int(NS/2))
    gcl = cls[end]
    cls = vcat(cls, reverse(cls))

    dcl = distinguishable_colors(1, [RGB(1,1,1), RGB(0,0,0), Plots.Colors.colormap("greens", 2)[2]], dropseed=true)

    p = plot(legend = :bottomright, xlabel = "time t", ylabel = "trajectory x(t)")
    for i in 1:NS
        plot!(p, evalpoints, qs[i, :]; color = cls[i], label = lbs[i], linewidth = 2)
    end

    plot!(p, evalpoints, m_in; label = "sample",
        color = :black, linewidth = 2.4)
    plot!(p, evalpoints, m_in .- s_in; label = "",
        color = :grey35, linewidth = 1.9, linestyle = :dash)
    plot!(p, evalpoints, m_in .+ s_in; label = "",
        color = :grey35, linewidth = 1.9, linestyle = :dash)
    
    plot!(p, th_expt, getfield.(xe_expt, :val); label = "PULL Euler ",
        color = dcl[1], linewidth = 2.4)
    plot!(p, th_expt, getfield.(xe_expt, :val) .- getfield.(xe_expt, :err); label = "",
        color = dcl[1], linewidth = 1.9, linestyle = :dash)
    plot!(p, th_expt, getfield.(xe_expt, :val) .+ getfield.(xe_expt, :err); label = "",
        color = dcl[1], linewidth = 1.9, linestyle = :dash)

    sliceloc = 2.5
    apidx = findfirst(th_expt .> sliceloc)
    # pidx = findfirst(evalpoints .> sliceloc)
    # kdest = kde(evalvals[pidx, :])
    nd = Normal(xe_expt[apidx].val, xe_expt[apidx].err)

    plot!(p, evalpoints[pidx]*ones(2), qs[[1,end], pidx]; label = "", color = :grey50)

    inletxlim = (1.3, 1.9)
    plot!(p, kdest; inset = (1, bbox(0.32,0.22,0.66,0.41)), subplot = 2,
        color = gcl, label = "", xlims = inletxlim,
        ylabel = "probability", xlabel = "state x($sliceloc)")
    plot!(p, nd; color = dcl[1], label = "", subplot = 2)

    # plot(kdest, color = :black, label = "sample", legend = :topright)
    # plot!(nd, color = dcl, label = "approx.")


    global p_dens = p

    plot(p, size = (333, 350))
end

# for the poster
p_fin = plot(p_model, p_dens; layout = (1,2), size =  (1000, 400).*0.65)

# savefig(p_fin, "nonl_plot.pdf")

p_fin

In [ ]:
p_dens_rs = plot(p_dens, size = (450, 370))
# savefig(p_dens_rs, "PULL-dens.pdf")
p_dens_rs

In [ ]:
# third plot, no buffer, and groot method
let 
    μ₀ = 0.6
    Σ₀ = 0.005
    lhwd = 2.2
    dcl = distinguishable_colors(3, [RGB(1,1,1), RGB(0,0,0)], dropseed=true)

    ylim_max = maximum(s_in)*1.03
    p = plot(xlabel = "time t", ylabel = "standard deviation", legend = :topright, ylim = (0.004, ylim_max))
    plot!(p, tspan, dzero.σ * ones(2), label = "fixed point", linestyle = :dot, color = :grey50, linewidth = lhwd)
    plot!(p, evalpoints, s_in; label = "sampling", linecolor = :black, linewidth = lhwd)
    plot!(p, th_expt, getfield.(xe_expt, :err); label = "PULL Euler (39)", linewidth = lhwd, 
        color = dcl[3])

    ex_lbl = ["", "h/2"]
    for (i, hi) in enumerate([hsp, hsp/2])
        th_bm, xe_bm = solvevareuler(μ₀ ± sqrt(Σ₀), tspan, hi)
        plot!(p, th_bm, getfield.(xe_bm, :err), label =  "MM (21) $(ex_lbl[i])",
            linewidth = lhwd, color = dcl[i])

        th_0b, xe_0b = linearized_euler(μ₀ ± sqrt(Σ₀), tspan, hi; lhist = 0 )
        plot!(p, th_0b, getfield.(xe_0b, :err), label =  "no cov PULL Euler $(ex_lbl[i])",
            linewidth = lhwd*1.5, linestyle = :dash, color = dcl[i])
    end

    global p_bm = p
    p
end

In [ ]:
p_bm_rs = plot(p_bm, size = (450, 280), legend = (0.51, 0.64))
# savefig(p_bm_rs, "PULL-bm.pdf")
p_bm_rs

In [ ]:
p_fin = plot(p_model, p_dens, p_bm; layout = (1,3), size =  (1000, 370))

In [ ]:
# savefig(p_fin, "nonlin_model_plot.pdf")

In [ ]:
bins = 0.3:0.01:1.8
function getdensity(data)
    h = fit(Histogram, data, bins)
    h = normalize(h, mode=:pdf)
    h.weights
end

let 
    # dens = [getdensity(evalval_row) for evalval_row in eachrow(evalvals)]
    # dens = reduce(hcat, dens)

    # # test = kde(dens)
    # contour(evalpoints, bins[1:end-1], dens; c = cgrad([:white, :darkgreen]), fill = false)

    # p = plot(legend = :bottomright, xlabel = "time t", ylabel = "trajectory x(t)")
    # heatmap!(p, evalpoints, bins[1:end-1], dens; c = cgrad([:white, :darkgreen]), colorbar=false)
    # plot!(p, evalpoints, m_in; label = "mean",
    #     color = :black, linewidth = 2.4)
    # plot!(p, evalpoints, m_in .- s_in; label = "Standard Deviation",
    #     color = :grey35, linewidth = 1.9)
    # plot!(p, evalpoints, m_in .+ s_in; label = "",
    #     color = :grey35, linewidth = 1.9)
    # global p_heat = p
end

# Solver

In [ ]:
# comparison solution 
f(x, p, t) = gp(x)
prob = ODEProblem(f, μ₀, tspan)

com_sol = solve(prob);

In [ ]:
h = 0.1
ylims = (0.04, 0.16)

special_h_idx = 4

b_csl = Plots.Colors.distinguishable_colors(5, [RGB(1,1,1), RGB(0,0,0)], dropseed=true)
s_csl = Plots.Colors.range(b_csl[special_h_idx], Plots.Colors.RGB(0,0,0), length = 4+2)
lhwd = 2.2
buffer_labels = [ "","","","buffer full"]
hv = [2*h, h, h/2, h/4]

p1 = plot(; xlabel = "time t", ylabel = "log mean error", legend = :bottomleft) 
p2 = plot(; 
    # title = "buffer = 200", 
    xlabel = "time t", ylabel = "standard deviation σ", 
    legend = :topright, ylims = ylims)
err = log.(abs.(com_sol(evalpoints).u .- m_in))
plot!(p1, evalpoints, err, label = "sampling", color = :black, linewidth = lhwd, linestyle = :dash)
plot!(p2, evalpoints, s_in; label = "", color = :black, linewidth = lhwd, linestyle = :dash)
plot!(p2, tspan, zeros(2); inset = (1, bbox(0.36,0.10,0.60,0.60)), subplot = 2,
    color = :black, linestyle = :dash, label = "",
    ylabel = "error", xlabel = "time t")

for (i, hi) in enumerate(hv)
    th_expt, xe_expt = linearized_euler(μ₀ ± sqrt(Σ₀), tspan, hi; lhist = 2000 )

    err = log.(abs.(com_sol(th_expt).u .- getfield.(xe_expt, :val)))
    plot!(p1, th_expt, err; label = "h = $hi", color = b_csl[i], linewidth = lhwd,)
    plot!(p2, th_expt, getfield.(xe_expt, :err); label = "", color = b_csl[i], 
        linewidth = lhwd,)

    interp = linear_interpolation(th_expt, getfield.(xe_expt, :err)) 
    err = abs.(s_in .- interp(evalpoints))
    plot!(p2, evalpoints, err; subplot = 2,
    color = b_csl[i], label = "")
    # scatter!(p2, [th_expt[200]], [getfield.(xe_expt, :err)[200]]; label = buffer_labels[i],
        # marker = :vline, markersize = 8, markerstrokewidth = 4, color =:grey35)
end

p_solver = plot(p1, p2; layout = (2, 1), size = (450, 480), bottom_margin = -5Plots.Measures.mm)

In [ ]:
# savefig(p_solver, "solver_conv.pdf")

In [ ]:
pt1 = plot(p2, p3)
p_sol = plot(pt1, p1; layout = (2, 1))

### Complete

In [ ]:
p_fin = plot(p_model, p_heat, p_sol; layout = (1,3), size = (1800, 570).*0.8)

In [ ]:
# savefig(p_fin, "solver_plot.pdf")

## Limitations
Problem when near unstable point

In [ ]:
μ₀ = 0.1
Σ₀ = 0.001

tspan = [0, 10]
lim_nevalpoints = 201
lim_evalpoints = range(tspan[1], tspan[2], length = lim_nevalpoints)

h = 0.01

nsols = 500 # missing 1
ninputsmpls = 70 # 150

function ens_samplepathsol()
    # pathsmpl = pathsample(fb, X, y)
    pathsmpl = exact_sample(xsample)
    f(x, p, t) = [pathsmpl(x[1])]
    prob = ODEProblem(f, [μ₀], tspan)

    function prob_func(prob, i, repeat)
        @. prob.u0 = μ₀ + randn()*sqrt(Σ₀)
        prob
    end
    
    ens_prob = EnsembleProblem(prob; prob_func)

    ens_sol = solve(ens_prob; trajectories = ninputsmpls, saveat = lim_evalpoints);
end

let 
    # enssols = [ens_samplepathsol() for i in 1:nsols];
    @time enssols = ThreadsX.map(x -> ens_samplepathsol(), 1:nsols);
    # global lim_evalpoints = range(tspan[1], tspan[2], length = 250)
    @time global lim_evalvals = reduce(vcat, map( ens -> map(x -> reduce(vcat, getfield(x, :u)), ens), enssols))

    global lim_m_in = [mean(getindex.(lim_evalvals, i)) for i in 1:lim_nevalpoints]
    global lim_s_in = [std(getindex.(lim_evalvals, i)) for i in 1:lim_nevalpoints]

    p1 = plot(lim_evalpoints, lim_m_in, ribbons = lim_s_in,
    color = :mediumseagreen, label = "summary", linewidth = lwd, legend = :bottomright)
    p2 = plot(lim_evalpoints, lim_s_in; label = "std", xlabel = "t")

    plot(p1, p2)
end

th_lim, xe_lim = linearized_euler(μ₀ ± sqrt(Σ₀), tspan, h/2; lhist = 700 )
maximum(getfield.(xe_lim, :err))

In [ ]:
plot(th_lim, getfield.(xe_lim, :val), ribbons = getfield.(xe_lim, :err))

In [ ]:
bins = -1.9:0.005:1.9 
function getdensity(data)
    h = fit(Histogram, data, bins)
    h = normalize(h, mode=:pdf)
    h.weights
end

dens = [getdensity(getindex.(lim_evalvals, i)) for i in 1:lim_nevalpoints];
dens = reduce(hcat, dens);

In [ ]:
bins = -1.9:0.005:1.9 
function getdensity(data)
    h = fit(Histogram, data, bins)
    h = normalize(h, mode=:pdf)
    h.weights
end

let 
    b_csl = Plots.Colors.distinguishable_colors(5+2, [RGB(1,1,1), RGB(0,0,0)])[3:end]

    dens = [getdensity(getindex.(lim_evalvals, i)) for i in 1:lim_nevalpoints]
    dens = reduce(hcat, dens)
    
    p = contour(lim_evalpoints, bins[1:end-1], dens; levels = 100, c = cgrad([:white, b_csl[3]], [0.20, 0.65]), colorbar=:none, linewidth = 3.5, size = (450, 350),
    legend = :bottomleft, xlabel = "time t", ylabel = "trajectory x(t)")


    
    plot!(p, lim_evalpoints, lim_m_in; label = "mean",
        color = :black, linewidth = 2.4)
    plot!(p, lim_evalpoints, lim_m_in .- lim_s_in; label = "std. deviation", linestyle = :dash,
        color = :grey35, linewidth = 1.9)
    plot!(p, lim_evalpoints, lim_m_in .+ lim_s_in; label = "", linestyle = :dash,
        color = :grey35, linewidth = 1.9)

    plot!(p, th_lim, getfield.(xe_lim, :val),
        color = b_csl[1], label = "PULL Euler", linewidth = 2.3)
    plot!(p, th_lim, getfield.(xe_lim, :val) .- getfield.(xe_lim, :err), linestyle = :dash,
        color = b_csl[1], label = "std. deviation", linewidth = 1.9)
    plot!(p, th_lim, getfield.(xe_lim, :val) .+ getfield.(xe_lim, :err), linestyle = :dash,
        color = b_csl[1], label = "", linewidth = 1.9)

    global p_heat_lim = p
end
2;
p_heat_lim

In [ ]:
# savefig(p_heat_lim, "limit.pdf")

In [ ]:
# for the poster
bins = -1.9:0.005:1.9 
function getdensity(data)
    h = fit(Histogram, data, bins)
    h = normalize(h, mode=:pdf)
    h.weights
end

let 
    b_csl = Plots.Colors.distinguishable_colors(5+2, [RGB(1,1,1), RGB(0,0,0)])[3:end]

    dens = [getdensity(getindex.(lim_evalvals, i)) for i in 1:lim_nevalpoints]
    dens = reduce(hcat, dens)

    # p = plot(legend = :topright, xlabel = "time t", ylabel = "trajectory x(t)",
        # size = (750, 550))
    # heatmap!(p, lim_evalpoints, bins[1:end-1], dens; c = cgrad([:white, b_csl[3]]), colorbar=false)
    p = contour(lim_evalpoints, bins[1:end-1], dens; levels = 100, c = cgrad([:white, b_csl[3]], [0.20, 0.65]), colorbar=:none, linewidth = 3.5, size = (450, 350),
    legend = :bottomleft, xlabel = "time t", ylabel = "trajectory x(t)")


    
    plot!(p, lim_evalpoints, lim_m_in; label = "sampled",
        color = :black, linewidth = 2.4)
    plot!(p, lim_evalpoints, lim_m_in .- lim_s_in; label = "", linestyle = :dash,
        color = :grey35, linewidth = 1.9)
    plot!(p, lim_evalpoints, lim_m_in .+ lim_s_in; label = "", linestyle = :dash,
        color = :grey35, linewidth = 1.9)

    plot!(p, th_lim, getfield.(xe_lim, :val),
        color = b_csl[1], label = "PULL Euler", linewidth = 2.3)
    plot!(p, th_lim, getfield.(xe_lim, :val) .- getfield.(xe_lim, :err), linestyle = :dash,
        color = b_csl[1], label = "", linewidth = 1.9)
    plot!(p, th_lim, getfield.(xe_lim, :val) .+ getfield.(xe_lim, :err), linestyle = :dash,
        color = b_csl[1], label = "", linewidth = 1.9)

    plot!(p, [-1., -2.], [1., 1.]; ribbons = [1., 1.], color = b_csl[3], label = "density", xlims = (0., 10.), ylims = (-2.6, 2.5))

    global p_heat_lim = p
end
2;
# plot!(p_heat_lim, size = (490, 350).*0.65)
plot!(p_heat_lim, size = (1000, 590).*0.35)

In [ ]:
# savefig(p_heat_lim, "limit.pdf")

In [ ]:
p_fin = plot(p1, p2, p_heat_lim; layout = (1,3), size = (1000, 370) ) 

In [ ]:
# savefig(p_fin, "cov-limit.pdf")